In [2]:
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader, TensorDataset

# -----------------------------
# 1️⃣ Load dataset
# -----------------------------
data = []
with open("crops.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line.strip()))

df = pd.DataFrame(data)

feature_cols = ["N", "P", "K", "temperature", "humidity", "ph", "rainfall"]
X = df[feature_cols].values.astype(np.float32)

y_raw = df["label"].apply(lambda x: x.split(","))
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_raw).astype(np.float32)

# -----------------------------
# 2️⃣ Train/Test split
# -----------------------------
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train)
Y_train_tensor = torch.from_numpy(Y_train)
X_val_tensor = torch.from_numpy(X_val)
Y_val_tensor = torch.from_numpy(Y_val)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# -----------------------------
# 3️⃣ Define MLP Model
# -----------------------------
class CropMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CropMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()  # for multi-label classification
        )

    def forward(self, x):
        return self.model(x)

input_dim = X_train.shape[1]
output_dim = Y_train.shape[1]
model = CropMLP(input_dim, output_dim)

# -----------------------------
# 4️⃣ Loss and optimizer
# -----------------------------
criterion = nn.BCELoss()  # Binary cross-entropy for multi-label
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 5️⃣ Training loop
# -----------------------------
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            outputs = model(xb)
            loss = criterion(outputs, yb)
            val_loss += loss.item() * xb.size(0)
    val_loss /= len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

# -----------------------------
# 6️⃣ Save model
# -----------------------------
torch.save(model.state_dict(), "crop.pth")
print("✅ Model saved as crop_mlp_model.pth")

# -----------------------------
# 7️⃣ Prediction function
# -----------------------------
def predict_crops_nn(model, mlb, env_features, top_n=10):
    model.eval()
    env_features = torch.tensor(env_features, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        probs = model(env_features).numpy().flatten()
    top_indices = probs.argsort()[-top_n:][::-1]
    return [mlb.classes_[i] for i in top_indices]

# -----------------------------
# 8️⃣ Test examples
# -----------------------------
test_envs = [
    [60, 40, 70, 26, 75, 6.3, 180],
    [10, 5, 5, 20, 60, 5.5, 100],
    [80, 60, 90, 30, 80, 6.8, 200]
]

for i, env in enumerate(test_envs):
    top_crops = predict_crops_nn(model, mlb, env, top_n=10)
    print(f"\n🌱 Environment {i+1}: {env}")
    print("Top predicted crops:", top_crops)


Epoch 1/50 - Train Loss: 2.5234 - Val Loss: 0.2015
Epoch 2/50 - Train Loss: 0.1929 - Val Loss: 0.1672
Epoch 3/50 - Train Loss: 0.1403 - Val Loss: 0.1173
Epoch 4/50 - Train Loss: 0.0976 - Val Loss: 0.0749
Epoch 5/50 - Train Loss: 0.0646 - Val Loss: 0.0616
Epoch 6/50 - Train Loss: 0.0531 - Val Loss: 0.0485
Epoch 7/50 - Train Loss: 0.0382 - Val Loss: 0.0327
Epoch 8/50 - Train Loss: 0.0297 - Val Loss: 0.0550
Epoch 9/50 - Train Loss: 0.0319 - Val Loss: 0.0245
Epoch 10/50 - Train Loss: 0.0193 - Val Loss: 0.0229
Epoch 11/50 - Train Loss: 0.0218 - Val Loss: 0.0256
Epoch 12/50 - Train Loss: 0.0179 - Val Loss: 0.0222
Epoch 13/50 - Train Loss: 0.0165 - Val Loss: 0.0252
Epoch 14/50 - Train Loss: 0.0190 - Val Loss: 0.0220
Epoch 15/50 - Train Loss: 0.0158 - Val Loss: 0.0225
Epoch 16/50 - Train Loss: 0.0159 - Val Loss: 0.0226
Epoch 17/50 - Train Loss: 0.0172 - Val Loss: 0.0312
Epoch 18/50 - Train Loss: 0.0214 - Val Loss: 0.0542
Epoch 19/50 - Train Loss: 0.0247 - Val Loss: 0.0354
Epoch 20/50 - Train L

In [3]:
torch.save(model.state_dict(), "crop.pth")


In [5]:
import joblib
joblib.dump(mlb, "mlb.pkl")


['mlb.pkl']

In [6]:
import torch
import torch.nn as nn
import joblib

# Load mlb
mlb = joblib.load("mlb.pkl")

# Rebuild model with same dimensions
input_dim = 7  # N, P, K, temp, humidity, ph, rainfall
output_dim = len(mlb.classes_)

class CropMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CropMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = CropMLP(input_dim, output_dim)
model.load_state_dict(torch.load("crop.pth", map_location=torch.device('cpu')))
model.eval()


CropMLP(
  (model): Sequential(
    (0): Linear(in_features=7, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=2698, bias=True)
    (5): Sigmoid()
  )
)

In [8]:
def predict_crops_nn(model, mlb, env_features, top_n=10):
    model.eval()
    env_features = torch.tensor(env_features, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        probs = model(env_features).numpy().flatten()
    top_indices = probs.argsort()[-top_n:][::-1]
    return [mlb.classes_[i] for i in top_indices]


In [9]:
import joblib
from google.colab import files
import torch

# ✅ Save MultiLabelBinarizer
joblib.dump(mlb, 'mlb.pkl')

# ✅ Save PyTorch model (state_dict)
torch.save(model.state_dict(), 'crop.pth')

# ✅ Download both files to local disk
files.download('mlb.pkl')
files.download('crop.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SyntaxError: invalid syntax (ipython-input-1216793997.py, line 1)

In [11]:
# -----------------------------
# offline_crop_predict.py
# -----------------------------

import joblib
import torch
import torch.nn as nn
import numpy as np

# -----------------------------
# 1️⃣ Define your MLP model architecture
# -----------------------------
class CropMLP(nn.Module):
    def __init__(self, input_dim=7, output_dim=2698):
        super(CropMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()  # for multi-label classification
        )

    def forward(self, x):
        return self.model(x)

# -----------------------------
# 2️⃣ Load saved model and MultiLabelBinarizer
# -----------------------------
# Load mlb
mlb = joblib.load('mlb.pkl')

# Load model
model = CropMLP(input_dim=7, output_dim=len(mlb.classes_))
model.load_state_dict(torch.load('crop.pth', map_location=torch.device('cpu')))
model.eval()

# -----------------------------
# 3️⃣ Prediction function
# -----------------------------
def predict_crops(env_features, top_n=10):
    env_tensor = torch.tensor(env_features, dtype=torch.float32).unsqueeze(0)  # shape: [1, 7]
    with torch.no_grad():
        probs = model(env_tensor).numpy().flatten()
    top_indices = probs.argsort()[-top_n:][::-1]
    return [mlb.classes_[i] for i in top_indices]

# -----------------------------
# 4️⃣ Example usage
# -----------------------------
if __name__ == "__main__":
    test_envs = [
        [60, 40, 70, 26, 75, 6.3, 180],
        [10, 5, 5, 20, 60, 5.5, 100],
        [80, 60, 90, 30, 80, 6.8, 200]
    ]

    for i, env in enumerate(test_envs):
        top_crops = predict_crops(env, top_n=5)
        print(f"\n🌱 Environment {i+1}: {env}")
        print("Top predicted crops:", top_crops)



🌱 Environment 1: [60, 40, 70, 26, 75, 6.3, 180]
Top predicted crops: ['ocotillo_fouquieria', 'pepper_bell', 'okra_clemson', 'cilantro_santo', 'tomato_cherokee']

🌱 Environment 2: [10, 5, 5, 20, 60, 5.5, 100]
Top predicted crops: ['pineapple', 'banana', 'pepper_jalapeno', 'guava', 'pineapple_smoothcayenne']

🌱 Environment 3: [80, 60, 90, 30, 80, 6.8, 200]
Top predicted crops: ['ocotillo_fouquieria', 'pomegranate_wonderful', 'yucca_soaptree', 'okra_clemson', 'pepper_bell']


In [12]:
# -----------------------------
# async_fastapi_crop_api.py
# -----------------------------
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import torch.nn as nn
import joblib
import numpy as np
import asyncio

# -----------------------------
# 1️⃣ Define MLP model
# -----------------------------
class CropMLP(nn.Module):
    def __init__(self, input_dim=7, output_dim=2698):
        super(CropMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# -----------------------------
# 2️⃣ Load saved model and MultiLabelBinarizer
# -----------------------------
mlb = joblib.load("mlb.pkl")
model = CropMLP(input_dim=7, output_dim=len(mlb.classes_))
model.load_state_dict(torch.load("crop.pth", map_location=torch.device("cpu")))
model.eval()

# -----------------------------
# 3️⃣ FastAPI app
# -----------------------------
app = FastAPI(title="Async Crop Prediction API")

# Input schema
class Environment(BaseModel):
    N: float
    P: float
    K: float
    temperature: float
    humidity: float
    ph: float
    rainfall: float
    top_n: int = 5  # optional, default top 5 crops

# -----------------------------
# 4️⃣ Asynchronous prediction function
# -----------------------------
async def async_predict_crops(env_features, top_n=5):
    # Simulate asynchronous behavior for heavy computation
    await asyncio.sleep(0)  # yield control to event loop
    env_tensor = torch.tensor(env_features, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        probs = model(env_tensor).numpy().flatten()
    top_indices = probs.argsort()[-top_n:][::-1]
    return [mlb.classes_[i] for i in top_indices]

# -----------------------------
# 5️⃣ Prediction endpoint
# -----------------------------
@app.post("/predict")
async def predict(env: Environment):
    features = [
        env.N, env.P, env.K,
        env.temperature, env.humidity,
        env.ph, env.rainfall
    ]
    crops = await async_predict_crops(features, top_n=env.top_n)
    return {"environment": features, "predicted_crops": crops}


In [13]:
!pip install pyngrok
from pyngrok import ngrok

# Expose port 8000 (where FastAPI runs)
public_url = ngrok.connect(8000)
print("Your public URL:", public_url)


ERROR:pyngrok.process.ngrok:t=2025-10-07T20:28:13+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-07T20:28:13+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-07T20:28:13+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.